In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tensorflow-addons
!pip install pyyaml h5py
!pip install toolz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 8.9 MB/s eta 0:00:00


In [4]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.12.0


## Setup

In [5]:
import os
# os.environ["MODIN_CPUS"] = "8"
# from distributed import Client
# client = Client()
import numpy as np
import math
import re
import random
import shutil

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_addons as tfa
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet as efn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tensorflow.keras.constraints import Constraint
# import allel
from scipy.spatial.distance import squareform
%matplotlib inline
from toolz import interleave
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.model_selection import KFold,StratifiedKFold

print("Tensorflow version " + tf.__version__)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Tensorflow version 2.12.0


## Hardware Config

In [6]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = strategy.num_replicas_in_sync
# Number of computing cores, is 8 for a TPU V3-8
print(f'N_REPLICAS: {N_REPLICAS}')

Running on TPU  grpc://10.126.251.74:8470
N_REPLICAS: 8


## Prepare the data

In [7]:
# load genotype
genotypes = pd.read_csv('[data_dir]/genotype_full.txt', sep='\t', index_col=0)
genotypes[genotypes == -1] = 0

headers = genotypes.columns[:]

In [8]:
genotypes.head()

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
01_01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_02,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_03,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01_06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
X = genotypes
X.shape

(4390, 28220)

In [10]:
# hyperparameters
feature_size = X.shape[1]
inChannel = 3
learning_rate = 0.01
weight_decay = 0.00001
embed_dim = 64  # Embedding size for each token
num_heads = 8 # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
regularization_coef_l1 = 1e-4
batch_size = 20
dropout_rate = 0.25
chunk_size = 3000
flanking_region_range = chunk_size

## Convert to tensorflow dataset

In [11]:
@tf.function()
def add_attention_mask(X_sample, y_sample):
  depth = 3
  mask_size = tf.cast(X_sample.shape[0]*0.5, dtype=tf.int64)
  mask_idx = tf.reshape(tf.random.shuffle(tf.range(X_sample.shape[0]))[:mask_size], (-1, 1))
  updates = tf.math.add(tf.ones(shape=(mask_idx.shape[0]), dtype=tf.int64), 1)
  X_masked = tf.tensor_scatter_nd_update(X_sample, mask_idx, updates)

  return tf.one_hot(X_masked, depth), tf.one_hot(y_sample, depth-1)

In [12]:
def map_values_1(x):
  return 0 if (x == 0 or x == 1) else 1

def map_values_2(x):
  return 0 if (x == 0 or x == 2) else 1

def get_dataset(x, chunk_start, chunk_end, start_offset, end_offset, batch_size, training=True):
  AUTO = tf.data.AUTOTUNE

  new_chunk_end = x.shape[1]

  dataset = tf.data.Dataset.from_tensor_slices((x,
                                                x[:, start_offset:new_chunk_end-end_offset]))
  # # Add Attention Mask

  if training:
    dataset = dataset.shuffle(x.shape[0], reshuffle_each_iteration=True)
    dataset = dataset.repeat()

  # Add Attention Mask
  dataset = dataset.map(add_attention_mask, num_parallel_calls=AUTO, deterministic=False)

  # Prefetech to not map the whole dataset
  dataset = dataset.prefetch(AUTO)

  dataset = dataset.batch(batch_size, drop_remainder=True, num_parallel_calls=AUTO)

  return dataset

## Custom Layers


In [13]:
class SharedNet(tf.keras.Model):
    def __init__(self, conv1_n_filters, conv2_n_filters, conv1_kernel_size, conv2_kernel_size, fc_len, start_offset, end_offset, n_features_out):
        super(SharedNet, self).__init__()
        self.fc_len = fc_len
        self.start_offset = start_offset
        self.end_offset = end_offset
        self.n_features_out = n_features_out
        self.conv1 = layers.Conv1D(conv1_n_filters, kernel_size=conv1_kernel_size, strides=1, padding='same', activation=tf.nn.relu)
        self.pool1 = layers.MaxPooling1D(2)
        self.bn1 = layers.BatchNormalization()
        self.do1 = layers.Dropout(0.5)
        self.conv2 = layers.Conv1D(conv2_n_filters, kernel_size=conv2_kernel_size, strides=1, padding='same', activation=tf.nn.relu)
        self.pool2 = layers.MaxPooling1D(2)
        self.bn2 = layers.BatchNormalization()
        self.do2 = layers.Dropout(0.5)
        # self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(4*(inChannel-1), activation=tf.nn.relu)
        self.reshape = layers.Reshape((-1, inChannel-1))
        # self.fc1 = layers.Dense(inChannel-1, activation=tf.nn.relu)

    def build(self, input_shape):
      self.input_len = input_shape[1]
      # self.reshape = layers.Reshape((self.input_len, self.fc_len))
      # super(SharedNet, self).build(input_shape)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.pool2(x)
        # x = self.flatten(x)
        x = self.fc1(x)
        x = self.reshape(x)
        return x[:, self.start_offset:self.input_len-self.end_offset,]


class EachNet(tf.keras.Model):
    def __init__(self, num_classes):
        super(EachNet, self).__init__()
        self.fc1 = layers.Dense(num_classes, activation=tf.nn.softmax)
        self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs):
        x = self.dropout(inputs)
        x = self.fc1(x)
        return x

## Modules

In [14]:
def chunk_module(conv1_n_filters, conv2_n_filters,
                conv1_kernel_size, conv2_kernel_size,
                input_len, inChannel, fc_len=256,
                start_offset=0, end_offset=0):
  inputs = layers.Input(shape=(input_len, inChannel))
  n_features_out = input_len - start_offset - end_offset
  xa = SharedNet(conv1_n_filters, conv2_n_filters, conv1_kernel_size, conv2_kernel_size, fc_len, start_offset, end_offset, n_features_out)(inputs)
  xa_out1 = EachNet(inChannel-1)(xa)

  model = keras.Model(inputs=inputs, outputs=xa_out1)
  return model

## Model

In [15]:
class DEEP_HLA(keras.Model):
  def __init__(
      self, in_channel,
      conv1_n_filters, conv2_n_filters,
      conv1_kernel_size, conv2_kernel_size,
      chunk_size=chunk_size,
      activation=tf.nn.relu,
      dropout_rate=0.5,
      flanking_region_range=flanking_region_range):
    super(DEEP_HLA, self).__init__()
    self.in_channel = in_channel
    self.embed_dim = embed_dim
    self.conv1_n_filters = conv1_n_filters
    self.conv2_n_filters = conv2_n_filters
    self.conv1_kernel_size = conv1_kernel_size
    self.conv2_kernel_size = conv2_kernel_size
    self.chunk_size = chunk_size
    self.activation = activation
    self.dropout_rate = dropout_rate
    self.flanking_region_range = flanking_region_range

  def build(self, input_shape):
    self.chunk_starts = list(range(0, input_shape[1], self.chunk_size))
    self.chunk_ends = []
    for cs in self.chunk_starts:
      self.chunk_ends.append(min(cs+self.chunk_size, input_shape[1]))
    self.mask_starts = [max(0, cs-self.flanking_region_range) for cs in self.chunk_starts]
    self.mask_ends = [min(ce+self.flanking_region_range, input_shape[1]) for ce in self.chunk_ends]
    self.chunkers = [chunk_module(self.conv1_n_filters, self.conv2_n_filters,
                                  self.conv1_kernel_size, self.conv2_kernel_size,
                                  self.mask_ends[i] - self.mask_starts[i],
                                  self.in_channel,
                                  start_offset=cs - self.mask_starts[i],
                                  end_offset=self.mask_ends[i]-self.chunk_ends[i]) for i,cs in enumerate(self.chunk_starts)]
    self.concat_layer = layers.Concatenate(axis=-2)
    super(DEEP_HLA, self).build(input_shape)


  def call(self, inputs):
    chunks = [self.chunkers[i](inputs[:, self.mask_starts[i]:self.mask_ends[i]]) for i, chunker in enumerate(self.chunkers)]
    y = self.concat_layer(chunks)
    return y


In [16]:
def create_model(base_dim = 512):
  model =  DEEP_HLA(in_channel=inChannel,
      conv1_n_filters=base_dim, conv2_n_filters=base_dim//2,
      conv1_kernel_size=base_dim//2, conv2_kernel_size=base_dim//2)
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  model.compile(optimizer, loss='categorical_crossentropy', metrics=tf.keras.metrics.CategoricalAccuracy())
  return model


In [17]:
model = create_model()
model.build((1, feature_size, inChannel))
model.summary()

Model: "deep_hla"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (Functional)          (None, 3000, 2)           33953550  
                                                                 
 model_1 (Functional)        (None, 3000, 2)           33953550  
                                                                 
 model_2 (Functional)        (None, 3000, 2)           33953550  
                                                                 
 model_3 (Functional)        (None, 3000, 2)           33953550  
                                                                 
 model_4 (Functional)        (None, 3000, 2)           33953550  
                                                                 
 model_5 (Functional)        (None, 3000, 2)           33953550  
                                                                 
 model_6 (Functional)        (None, 3000, 2)           339

In [18]:
METRIC = "val_loss"

def create_callbacks(kfold=0, metric = METRIC):

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='auto',
        factor=0.5,
        patience=5,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='auto',
        patience= 20,
        verbose=1,
        restore_best_weights=True
    )

    callbacks = [
                 reducelr,
                 earlystop]

    return callbacks

## Training

In [19]:
save_dir = "./drive/MyDrive/ShiLab/YEAST/DEEP_HLA/"

if not os.path.exists(save_dir):
  # shutil.rmtree(save_dir)
  os.makedirs(save_dir)

In [20]:
# A TPU V3-8 has 8 computing cores, the global batch size will be 1/16 x 8 = 8/128
BATCH_SIZE_BASE = 4
# Training configuration
BATCH_SIZE = BATCH_SIZE_BASE * N_REPLICAS if TPU else 32
BATCH_SIZE

32

## Timing test

In [21]:
N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  if fold != 1:
    continue
  accuracies = []
  print(f"Training using fold {fold}")

  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)

  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=1,
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=1)
  for missing_perc in [ 0.1,
                        0.2
                        ]:
    save_name = save_dir + f"preds_mixed_mr_{missing_perc}_fold_{fold}_probs"
    avg_accuracy = []
    preds = []
    true_labels = []
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1], inChannel-1), dtype=np.float32)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)

    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # mMask = np.tile(attention_mask, (1, 1, 1))
      # predict
    predict_onehot = model.predict(test_X_missing, verbose=1)


Training using fold 1
82/82 [==============================] - 272s 2s/step - loss: 0.4042 - categorical_accuracy: 0.7398 - val_loss: 0.5340 - val_categorical_accuracy: 0.8125 - lr: 0.0100


100%|██████████| 1464/1464 [00:00<00:00, 4988.04it/s]


46/46 [==============================] - 79s 2s/step


100%|██████████| 1464/1464 [00:00<00:00, 3130.74it/s]


46/46 [==============================] - 33s 719ms/step


## Training loop

In [ ]:
N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  if fold != 3:
    continue
  accuracies = []
  print(f"Training using fold {fold}")

  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)

  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS,
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=1)
  for missing_perc in [
                        0.01,
                        0.05,
                        0.1,
                        0.2
                        ]:
    save_name = save_dir + f"preds_mixed_mr_{missing_perc}_fold_{fold}_probs"
    avg_accuracy = []
    preds = []
    true_labels = []
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1], inChannel-1), dtype=np.float32)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)

    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      predict_onehot = model.predict(test_X_missing[i:i+1], verbose=0)
      predict_missing_onehot = predict_onehot[:, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)

      preds.extend(predict_missing.ravel().tolist())

      predict_haplotypes = np.argmax(predict_onehot, axis=2)
      to_save_array[i] = predict_onehot[:]
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      # label_missing = np.argmax(label_missing_onehot, axis=2)
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      # print('{}/{}, accuracy: {:.4f}'.format(
      #     i, test_X_missing.shape[0], accuracy))

      avg_accuracy.append(accuracy)

    # df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    # df.to_csv(save_name)
    np.save(save_name, to_save_array)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))



Training using fold 3
Epoch 1/1000


In [ ]:
from google.colab import runtime
runtime.unassign()

In [ ]:
N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  if fold != 2:
    continue
  accuracies = []
  print(f"Training using fold {fold}")

  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)

  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS,
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=1)
  for missing_perc in [
                        0.01,
                        0.05,
                        0.1,
                        0.2
                        ]:
    save_name = save_dir + f"preds_mixed_mr_{missing_perc}_fold_{fold}_probs"
    avg_accuracy = []
    preds = []
    true_labels = []

    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # mMask = np.tile(attention_mask, (1, 1, 1))
      # predict
    predict_onehot = model.predict(test_X_missing, verbose=1)
    to_save_array = np.copy(predict_onehot)
      # predict_onehot = model.predict([test_X_missing[i:i + 1, :, :], mMask])
      # only care the missing position
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      predict_missing_onehot = predict_onehot[i:i+1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)

      preds.extend(predict_missing.ravel().tolist())

      predict_haplotypes = np.argmax(predict_onehot[i:i+1, :, :], axis=2)
      # to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      # label_missing = np.argmax(label_missing_onehot, axis=2)
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      # print('{}/{}, accuracy: {:.4f}'.format(
      #     i, test_X_missing.shape[0], accuracy))

      avg_accuracy.append(accuracy)

    # df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    # df.to_csv(save_name)
    np.save(save_name, to_save_array)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))



In [ ]:
N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  if fold != 1:
    continue
  accuracies = []
  print(f"Training using fold {fold}")

  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)

  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS,
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=1)
  for missing_perc in [
                        0.01,
                        0.05,
                        0.1,
                        0.2
                        ]:
    save_name = save_dir + f"preds_mixed_mr_{missing_perc}_fold_{fold}_probs"
    avg_accuracy = []
    preds = []
    true_labels = []

    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # mMask = np.tile(attention_mask, (1, 1, 1))
      # predict
    predict_onehot = model.predict(test_X_missing, verbose=1)
    to_save_array = np.copy(predict_onehot)
      # predict_onehot = model.predict([test_X_missing[i:i + 1, :, :], mMask])
      # only care the missing position
    for i in tqdm(list(range(test_dataset.shape[0]))):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      predict_missing_onehot = predict_onehot[i:i+1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)

      preds.extend(predict_missing.ravel().tolist())

      predict_haplotypes = np.argmax(predict_onehot[i:i+1, :, :], axis=2)
      # to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      # label_missing = np.argmax(label_missing_onehot, axis=2)
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      # print('{}/{}, accuracy: {:.4f}'.format(
      #     i, test_X_missing.shape[0], accuracy))

      avg_accuracy.append(accuracy)

    # df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    # df.to_csv(save_name)
    np.save(save_name, to_save_array)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))

